# **라이브러리**

In [1]:
import pandas as pd
import numpy as np

from imblearn.under_sampling import *
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
import warnings
warnings.filterwarnings(action='ignore')

import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'malgun gothic'

# **함수모음**

## **모델링**

In [2]:
def modeling(model, X, y, test):    
    model.fit(X, y)
    pred = model.predict_proba(test)
    pred = (pred[:,0] < 0.5)*1         # threshold 부도기업일 확률이 10% 이상이면 부도로 판단해라.
    return pred

def modeling_(model, X, y, test):    
    model.fit(X, y)
    pred = model.predict_proba(test)         # threshold 부도기업일 확률이 10% 이상이면 부도로 판단해라.
    return pred

######################################## Split train, test
def splittrain(data):
    subdata = data.sort_values(by='년').reset_index(drop=True)
    data = subdata.loc[subdata['년'] < 2015]
    return data

def splittesst(data):
    global test
    subdata = data.sort_values(by='년').reset_index(drop=True)
    test = subdata.loc[subdata['년'] >= 2015]
    return test

## **스코어**

In [3]:
def scoring(y_true, y_pred):
    print('accuracy:', accuracy_score(y_true,y_pred))
    print('f1-Score:', f1_score(y_true, y_pred, average='macro'))
    print('Recall:',recall_score(y_true, y_pred, average='macro'))
    print('Precision:',precision_score(y_true, y_pred, average='macro'))
    print(confusion_matrix(y_true, y_pred))

## **귀찮다 귀찮아**

# **데이터 나누기**

## **재무데이터**

In [24]:
dataset = pd.read_csv(r'C:\Users\JH\Desktop\Final\Ubion-CorporateBankruptcyAnalysis\Code\JH\data\0y_finan.csv')
dataset1 = pd.read_csv(r'C:\Users\JH\Desktop\Final\Ubion-CorporateBankruptcyAnalysis\Code\JH\data\1y_finan.csv')
dataset2 = pd.read_csv(r'C:\Users\JH\Desktop\Final\Ubion-CorporateBankruptcyAnalysis\Code\JH\data\2y_finan.csv')
dataset3 = pd.read_csv(r'C:\Users\JH\Desktop\Final\Ubion-CorporateBankruptcyAnalysis\Code\JH\data\3y_finan.csv')
dataset4 = pd.read_csv(r'C:\Users\JH\Desktop\Final\Ubion-CorporateBankruptcyAnalysis\Code\JH\data\4y_finan.csv')

dataset.columns

Index(['회사명', '거래소코드', '회계년도', '설비투자효율', '총자본투자효율', '부채구성비율', '비유동장기적합률',
       '재고자산보유기간', '매출채권회수기간', '매입채무회전률', '경영자본회전률', '경영자본순이익률', '자기자본순이익률',
       '자기자본증가율', '년', 'key', '부도'],
      dtype='object')

## **재무데이터 + 비재무데이터**

In [25]:
ajdataset = pd.read_csv(r'C:\Users\JH\Desktop\Final\Ubion-CorporateBankruptcyAnalysis\Code\SH\csv\step3\dataset_step3.csv',index_col=0)
ajdataset1 = pd.read_csv(r'C:\Users\JH\Desktop\Final\Ubion-CorporateBankruptcyAnalysis\Code\SH\csv\step3\dataset1_step3.csv',index_col=0)
ajdataset2 = pd.read_csv(r'C:\Users\JH\Desktop\Final\Ubion-CorporateBankruptcyAnalysis\Code\SH\csv\step3\dataset2_step3.csv',index_col=0)
ajdataset3 = pd.read_csv(r'C:\Users\JH\Desktop\Final\Ubion-CorporateBankruptcyAnalysis\Code\SH\csv\step3\dataset3_step3.csv',index_col=0)
ajdataset4 = pd.read_csv(r'C:\Users\JH\Desktop\Final\Ubion-CorporateBankruptcyAnalysis\Code\SH\csv\step3\dataset4_step3.csv',index_col=0)

drop_feature = ['회사명', '거래소코드', '회계년도','년', 'key', '부도']

ajdataset.columns

Index(['회사명', '거래소코드', '회계년도', '설비투자효율', '총자본투자효율', '부채구성비율', '비유동장기적합률',
       '재고자산보유기간', '매출채권회수기간', '매입채무회전률', '경영자본회전률', '경영자본순이익률', '자기자본순이익률',
       '자기자본증가율', '년', 'key', '부도', '대표이사_변경', '최대주주_변경', '회계처리위반', '횡령배임',
       '영업조업중단', '종가변동률', '출자목적_투자', '출자목적_경영권', '출자목적_영업이익', '신종채권_운영',
       '신종채권_시설', '신종채권_기타'],
      dtype='object')

In [26]:
dataset_ = splittrain(dataset)
test_ = splittesst(dataset)
dataset1_ = splittrain(dataset1)
test1_ = splittesst(dataset1)
dataset2_ = splittrain(dataset2)
test2_ = splittesst(dataset2)
dataset3_ = splittrain(dataset3)
test3_ = splittesst(dataset3)
dataset4_ = splittrain(dataset4)
test4_ = splittesst(dataset4)

ajdataset_ = splittrain(ajdataset)
ajtest_ = splittesst(ajdataset)
ajdataset1_ = splittrain(ajdataset1)
ajtest1_ = splittesst(ajdataset1)
ajdataset2_ = splittrain(ajdataset2)
ajtest2_ = splittesst(ajdataset2)
ajdataset3_ = splittrain(ajdataset3)
ajtest3_ = splittesst(ajdataset3)
ajdataset4_ = splittrain(ajdataset4)
ajtest4_ = splittesst(ajdataset4)


## **세트 1. 재무데이터 train,test 분리**

In [27]:
X = dataset_.drop(drop_feature, axis=1)
X1 = dataset1_.drop(drop_feature, axis=1)
X2 = dataset2_.drop(drop_feature, axis=1)
X3 = dataset3_.drop(drop_feature, axis=1)
X4 = dataset4_.drop(drop_feature, axis=1)

y = dataset_['부도']
y1 = dataset1_['부도']
y2 = dataset2_['부도']
y3 = dataset3_['부도']
y4 = dataset4_['부도']

X_test =test_.drop(drop_feature, axis=1)
X1_test =test1_.drop(drop_feature, axis=1)
X2_test =test2_.drop(drop_feature, axis=1)
X3_test =test3_.drop(drop_feature, axis=1)
X4_test =test4_.drop(drop_feature, axis=1)

y_test= test_['부도']
y1_test =test1_['부도']
y2_test =test2_['부도']
y3_test =test3_['부도']
y4_test =test4_['부도']

print(X_test.shape, y_test.shape)


(8504, 11) (8504,)


## **세트2 재무 + 비재무데이터 train, test 분리**

In [28]:
ajX = ajdataset_.drop(drop_feature, axis=1)
ajX1 = ajdataset1_.drop(drop_feature, axis=1)
ajX2 = ajdataset2_.drop(drop_feature, axis=1)
ajX3 = ajdataset3_.drop(drop_feature, axis=1)
ajX4 = ajdataset4_.drop(drop_feature, axis=1)

ajy = ajdataset_['부도']
ajy1 = ajdataset1_['부도']
ajy2 = ajdataset2_['부도']
ajy3 = ajdataset3_['부도']
ajy4 = ajdataset4_['부도']

ajX_test =ajtest_.drop(drop_feature, axis=1)
ajX1_test =ajtest1_.drop(drop_feature, axis=1)
ajX2_test =ajtest2_.drop(drop_feature, axis=1)
ajX3_test =ajtest3_.drop(drop_feature, axis=1)
ajX4_test =ajtest4_.drop(drop_feature, axis=1)

ajy_test= ajtest_['부도']
ajy1_test =ajtest1_['부도']
ajy2_test =ajtest2_['부도']
ajy3_test =ajtest3_['부도']
ajy4_test =ajtest4_['부도']

print(ajX_test.shape, ajy_test.shape)


(8504, 23) (8504,)


# **YEAR**

# **데이터 검증 및 언더샘플링**

In [9]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

mdn = ['Decision Tree','Random Forest','Adaboost','logistic', 'KNN','SVC','lightGBM','Catboost','MLP','Xgboost']

def run_skfold(feature, target):
    X, y = RandomUnderSampler(random_state=1).fit_resample(feature, target) # RandomUnderSampling
    skf = StratifiedKFold(n_splits=4, shuffle=True)
    arr = [] # Validation Set's Accuracy
    models = [DecisionTreeClassifier(),RandomForestClassifier(),AdaBoostClassifier(),LogisticRegression(),KNeighborsClassifier(),
    SVC(probability=True),LGBMClassifier(),CatBoostClassifier(silent=True),MLPClassifier(),XGBClassifier()]

    for model in models:
        val_pred = 0
        for train_index, val_index in skf.split(X,y):
            # print("TRAIN:", train_index, "TEST:", val_index)
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]
            val_pred += accuracy_score(y_val, modeling(model, X_train, y_train, X_val))
        arr.append(val_pred/4)
    return arr # each models accuracy

In [33]:
# 1,2,3,4 숫자구분해줘야함

X = np.array(X4)
ajX = np.array(ajX4)

y = y4
ajy = ajy4

y_test = y4_test
ajy_test = ajy4_test

vanval = run_skfold(X, y)
ajval = run_skfold(ajX,ajy)

[17:48:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:48:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:48:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:48:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

## **테스트 셋 검증**

In [34]:
X, y = RandomUnderSampler(random_state=1).fit_resample(X, y)
########################################################################################## 데이터셋 1
dt_clf = modeling(DecisionTreeClassifier(), X, y, X_test)
rf_clf = modeling(RandomForestClassifier(),X, y, X_test)
ada_clf = modeling(AdaBoostClassifier(), X, y, X_test)
lg_clf = modeling(LogisticRegression(), X, y, X_test)
knn_clf = modeling(KNeighborsClassifier(), X, y, X_test)
svc_clf = modeling(SVC(probability=True), X, y, X_test)
lgmb_clf = modeling(LGBMClassifier(), X, y, X_test)
cat_clf = modeling(CatBoostClassifier(silent=True),X, y, X_test)
mlp_clf = modeling(MLPClassifier(),X, y, X_test)
xgb_clf = modeling(XGBClassifier(),X, y, X_test)

ajX, ajy = RandomUnderSampler(random_state=1).fit_resample(ajX, ajy)
########################################################################################## 데이터셋 2
ajdt_clf = modeling(DecisionTreeClassifier(), ajX, ajy, ajX_test)
ajrf_clf = modeling(RandomForestClassifier(),ajX, ajy, ajX_test)
ajada_clf = modeling(AdaBoostClassifier(), ajX, ajy, ajX_test)
ajlg_clf = modeling(LogisticRegression(), ajX, ajy, ajX_test)
ajknn_clf = modeling(KNeighborsClassifier(), ajX, ajy, ajX_test)
ajsvc_clf = modeling(SVC(probability=True), ajX, ajy, ajX_test)
ajlgmb_clf = modeling(LGBMClassifier(), ajX, ajy, ajX_test)
ajcat_clf = modeling(CatBoostClassifier(silent=True),ajX, ajy, ajX_test)
ajmlp_clf = modeling(MLPClassifier(),ajX, ajy, ajX_test)
ajxgb_clf = modeling(XGBClassifier(),ajX, ajy, ajX_test)


[17:48:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:48:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [21]:
for i in arr:
    print(scoring(y_test , i))
    print('-'*60)

for i in ajarr:
    print(scoring(y_test , i))
    print('-'*60)

accuracy: 0.5727892756349953
f1-Score: 0.3672241540008709
Recall: 0.5591573599708473
Precision: 0.5006237140735128
[[4859 3623]
 [  10   12]]
None
------------------------------------------------------------
accuracy: 0.6575729068673566
f1-Score: 0.4006193139835149
Recall: 0.6016591284216843
Precision: 0.5011645710118201
[[5580 2902]
 [  10   12]]
None
------------------------------------------------------------
accuracy: 0.6559266227657573
f1-Score: 0.3993551784006103
Recall: 0.5554972026323123
Precision: 0.5006347175029051
[[5568 2914]
 [  12   10]]
None
------------------------------------------------------------
accuracy: 0.7557619943555973
f1-Score: 0.43419437330731775
Recall: 0.5602077125892264
Precision: 0.5008432948079836
[[6419 2063]
 [  14    8]]
None
------------------------------------------------------------
accuracy: 0.6603951081843838
f1-Score: 0.40135419271774064
Recall: 0.5804055647253006
Precision: 0.5009250874036193
[[5605 2877]
 [  11   11]]
None
-------------------

In [35]:
arr =[dt_clf,rf_clf,ada_clf,lg_clf,knn_clf,svc_clf,lgmb_clf,cat_clf,mlp_clf,xgb_clf]
ajarr = [ajdt_clf,ajrf_clf,ajada_clf,ajlg_clf,ajknn_clf,ajsvc_clf,ajlgmb_clf,ajcat_clf,ajmlp_clf,ajxgb_clf]

def test_acc(y_true, arr_):
    acc = []
    for i in arr_:
        acc.append(round(accuracy_score(y_true, i),4))
    return acc

ogntest = test_acc(y_test, arr)
ajdtest = test_acc(ajy_test, ajarr)

def get_f1(y_true, arr_):
    f1 = []
    for i in arr_:
        f1.append(round(f1_score(y_true, i, average='macro'),4))
    return f1
     
ognf1 = get_f1(y_test, arr)
ajdf1 = get_f1(ajy_test, ajarr)

def get_rc(y_true, arr_):
    rc = []
    for i in arr_:
        rc.append(round(recall_score(y_true, i, average='macro'),4))
    return rc

ognrc = get_rc(y_test, arr)
ajdrc = get_rc(ajy_test, ajarr)

def get_pc(y_true, arr_):
    pc = []
    for i in arr_:
        pc.append(round(precision_score(y_true, i, average='macro'),4))
    return pc

ognpc = get_pc(y_test, arr)
ajdpc = get_pc(ajy_test, ajarr)


In [36]:
subsetone = pd.DataFrame({'model': mdn, 'SET1 val Accuracy': vanval, 'SET1 test Accuracy' : ogntest, 'SET1 test f1-Score': ognf1, 'SET1 test Precision': ognpc, 'SET1 test Recall': ognrc, 
                                        'SET2 val Accuracy' : ajval, 'SET2 test Accuracy' : ajdtest, 'SET2 test f1_Score': ajdf1, 'SET2 test Precision': ajdpc, 'SET2 test Recall': ajdrc})
subsetone.to_csv('셋1vs셋2정확도비교및교차검증4년전.csv', index=False)